In [4]:
import polars as pl

sales_series = pl.Series("sales", [150.00, 300.00, 250.00])
sales_series

sales
f64
150.0
300.0
250.0


## Cast python list to polars enum series

In [22]:
reals_labels = ["real_negatives", "real_positives", "real_competing", "real_censored"]
censoring_assumption_labels = ["excluded", "adjusted"]
competing_assumption_labels = ["excluded", "adjusted_as_negative", "adjusted_as_censored"]

reals_enum = pl.Enum(reals_labels)
censoring_assumption_enum = pl.Enum(censoring_assumption_labels)
competing_assumption_enum = pl.Enum(competing_assumption_labels)

df_reals = pl.DataFrame({"reals": pl.Series(reals_labels, dtype=reals_enum)})
df_censoring_assumption = pl.DataFrame({"censoring_assumption": pl.Series(censoring_assumption_labels, dtype=censoring_assumption_enum)})
df_competing_assumption = pl.DataFrame({"competing_assumption": pl.Series(competing_assumption_labels, dtype=competing_assumption_enum)})


aj_data_combinations = (
    df_reals
    .join(df_censoring_assumption, how="cross")
    .join(df_competing_assumption, how="cross")
)

print(aj_data_combinations)

shape: (24, 3)
┌────────────────┬──────────────────────┬──────────────────────┐
│ reals          ┆ censoring_assumption ┆ competing_assumption │
│ ---            ┆ ---                  ┆ ---                  │
│ enum           ┆ enum                 ┆ enum                 │
╞════════════════╪══════════════════════╪══════════════════════╡
│ real_negatives ┆ excluded             ┆ excluded             │
│ real_negatives ┆ excluded             ┆ adjusted_as_negative │
│ real_negatives ┆ excluded             ┆ adjusted_as_censored │
│ real_negatives ┆ adjusted             ┆ excluded             │
│ real_negatives ┆ adjusted             ┆ adjusted_as_negative │
│ …              ┆ …                    ┆ …                    │
│ real_censored  ┆ excluded             ┆ adjusted_as_negative │
│ real_censored  ┆ excluded             ┆ adjusted_as_censored │
│ real_censored  ┆ adjusted             ┆ excluded             │
│ real_censored  ┆ adjusted             ┆ adjusted_as_negative │
│ real_cen

## Map numerics to Enums

In [ ]:
reals_map = {
    0: "real_negatives",
    1: "real_positives",
    2: "real_competing"
}

df = pl.DataFrame(
    {
        "country_code": [0, 0, 1, 2, 1],
    }
)

df


df.with_columns(
    pl.col("country_code").replace_strict(country_code_dict, return_dtype=reals_enum).alias("remapped")
)

country_code,remapped
i64,enum
0,"""real_negatives"""
0,"""real_negatives"""
1,"""real_positives"""
2,"""real_competing"""
1,"""real_positives"""
